In [9]:
import librosa
import matplotlib.pyplot as plt
import librosa.display 
import numpy as np
import sox
import os
import soundfile as sf
from itertools import groupby
import pandas as pd

In [19]:
path_parent=r"F:\DLR\matchedspeakersound\new_0730\text\12m"
folders_parent=os.listdir(path_parent)
folders_parent.sort()

In [20]:
def outputall(path,paths,folder):
    i=0 
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f: 
        lines = f.readlines()
        line_n=len(lines)
        startp=np.zeros((line_n,1))
        duration=np.zeros((line_n,1))
        who=[np.nan]*line_n
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f:
        for line in f:
            line=line.split(' ')
            startp[i]=line[3] #start of an utterance
            duration[i]=line[4] #duration of an utterance
            who[i]=line[7] #who is the speaker of an utterance
            i=i+1
    endtime=startp+duration # calculate the endtime of an utterance
    startend=np.c_[startp,endtime]
    for i in range(len(startend)): # Line labeled as 'SPEECH' is confused, remove them
        if who[i]=='SPEECH':
            startend[i,:]=np.nan
    startendclear=startend[~np.isnan(startend).all(axis=1), :]
    whoclear=[]
    for i in range(len(startend)):
        if who[i]!='SPEECH':
            whoclear.append(who[i])
            
    blocktime=np.c_[np.arange(6)*300+.001,np.arange(6)*300+300.001]
    
    count_t=np.zeros(len(blocktime))
    count_ttt=np.zeros(len(blocktime))
    for i in range(len(blocktime)):
        count=0
        for j in range(len(whoclear)):
            if whoclear[j]=='KCHI':
                if startendclear[j,0]>blocktime[i,0] and startendclear[j,1]<blocktime[i,1]:
                    count=count+1
        if count>20:
            count_ttt[i]=count
            count_t[i]=1

    highmoment=blocktime[np.where(count_t==1)[0]]
    
    count_ts=np.zeros(len(highmoment))
    count_tt=np.zeros(len(highmoment))
    
    for i in range(len(highmoment)):
        counts=0
        for j in range(len(whoclear)-1):
            if startendclear[j,0]>highmoment[i,0] and startendclear[j,1]<highmoment[i,1]:
                if whoclear[j]=='KCHI':
                    if whoclear[j+1]!='KCHI':
                        counts+=1
                if whoclear[j]!='KCHI':
                    if whoclear[j+1]=='KCHI':
                        counts+=1
        if counts>40:
            count_tt[i]=counts
            count_ts[i]=1

    highswitch=highmoment[np.where(count_ts==1)[0]]
        
    output=highmoment
    outputs=highswitch
    allsp=pd.DataFrame(output)
    allsps=pd.DataFrame(outputs)
    allsp.columns=['starthm','endhm']
    allsps.columns=['starths','endhs']
    allsp.to_csv(paths+'\\'+folder+'\\'+'highmoment.csv')
    allsps.to_csv(paths+'\\'+folder+'\\'+'highswitch.csv')

    allss=np.c_[startendclear,whoclear]
    allssp=pd.DataFrame(allss)
    allssp.columns=['Starttime','Endtime','Speaker']
    allssp.to_csv(paths+'\\'+folder+'\\'+'VTC.csv')

In [21]:
for ii in range(len(folders_parent)):
    path=path_parent+'\\'+folders_parent[ii] #path to the ALICE folder
    paths=path #save the outputs in the same folder or you could create new folders
    folders=os.listdir(path)
    folders.sort()
    for i in range(len(folders)):
        outputall(path,paths,folders[i])